In [61]:
import numpy as np

In [3]:
import pickle

single_run = None
with open('data/single_run.pkl', 'rb') as rf:
    single_run = pickle.load(rf)

with open('data/n_run.pkl', 'rb') as rf:
    n_runs = pickle.load(rf)

In [4]:
chromo_set_1, score_set_1 = single_run

In [62]:
n_results = np.array(n_runs)

/var/folders/ms/33mxkwtn255b353qtr2c5m4r0000gn/T/ipykernel_13232/228490561.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  n_results = np.array(n_runs)


# Collect Common indexes from res of single run evolution

In [19]:
from n_run_evolutions import logmodel, X_train, Y_train, X_test, Y_test, accuracy_score

In [22]:
score_set_1

[0.612,
 0.612,
 0.644,
 0.684,
 0.684,
 0.684,
 0.684,
 0.684,
 0.684,
 0.732,
 0.732,
 0.736,
 0.736,
 0.736,
 0.736,
 0.736,
 0.736,
 0.736,
 0.752,
 0.752,
 0.752,
 0.752,
 0.764,
 0.78,
 0.78,
 0.78,
 0.78,
 0.78,
 0.78,
 0.78]

In [23]:
chromo_len_in_each_gen = []
for i in chromo_set_1:
    chromo_len_in_each_gen.append(np.count_nonzero(i==1))
chromo_len_in_each_gen = np.array(chromo_len_in_each_gen)
chromo_len_in_each_gen

array([100, 100, 338, 480, 480, 480, 480, 480, 480, 375, 375, 666, 666,
       666, 666, 666, 666, 666, 534, 534, 534, 534, 576, 709, 709, 709,
       709, 709, 709, 709])

In [24]:
common_index = dict()
for chromo in chromo_set_1:
    for i in range(len(chromo)):
        if chromo[i] == 1:
            common_index[i] = common_index.setdefault(i, 0) + 1

sorted_common_index = np.array(sorted(common_index.items(), key=lambda x: x[1], reverse=True)[:200])
sorted_common_index[:, 0]

array([ 156,  594,  779,   81,  179,  220,  272,  723, 1190, 1343, 1389,
       1432,   13,  858,   79,  458,  517,  603, 1057, 1070, 1195, 1470,
        744,  869, 1376,  732,  902, 1100,  151,  167,  170,  422,  979,
       1182, 1248,  866, 1064, 1305, 1319,  564,  535,  565,  623,  929,
        149,  251,  261, 1033, 1465,  176,  207,  338,  361,  407,  419,
        432,  495,  515,  525,  532,  590,  713,  725,  750,  754, 1165,
       1180, 1246, 1321, 1375, 1510, 1157, 1467,  104,  113,  187,  288,
        523,  897, 1292, 1496,  938, 1000, 1317,  304,  519,  599,  656,
        693,  896,  915,  997, 1052, 1264,  101,  139,  198,  218,  253,
        279,  283,  313,  352,  790,  967, 1115, 1130, 1132, 1172, 1213,
       1255, 1294, 1330, 1335, 1354, 1380,   23,  102,  105,  490,  544,
        583,  619,  734, 1454,  126,  178,  186,  188,  355,  357,  425,
        868, 1023, 1096, 1120, 1129, 1136, 1137, 1140, 1204, 1262, 1309,
       1348, 1360, 1502,   33,   46,   65,  426,  4

In [25]:
logmodel.fit(X_train[:,sorted_common_index[:, 0]],Y_train)         
predictions = logmodel.predict(X_test[:,sorted_common_index[:, 0]])
accuracy_score(Y_test,predictions)

0.688

In [18]:
common_index = []
for chromo in chromo_set_1:
    index = []
    for i in range(len(chromo)):
        if chromo[i] == 1:
            index.append(i)
    common_index.extend(index)

print(len(set(common_index)))

logmodel.fit(X_train[:,common_index],Y_train)         
predictions = logmodel.predict(X_test[:,common_index])
accuracy_score(Y_test,predictions)

1380


0.78

# Collect Common indexes from res of n run evolution

In [82]:
chromo_len_best_gen = []
n_best_gen = []
common_index_n_runs = dict()

for run in n_results:
    best_gen = np.array(run[:, -1])
    n_best_gen.append(best_gen)
    chromo_len_best_gen.append(np.count_nonzero(best_gen[0]==1))
    # print(chromo_len_best_gen)
    # print(best_gen[0])
    for i in range(len(best_gen[0])):
        if best_gen[0][i] == 1:
            common_index_n_runs[i] = common_index_n_runs.setdefault(i, 0) + 1

sorted_common_index_n_runs = np.array(sorted(common_index_n_runs.items(), key=lambda x: x[1], reverse=True)[:200])
sorted_common_index_n_runs

array([[ 599,   30],
       [ 803,   30],
       [ 594,   28],
       [ 417,   25],
       [ 461,   24],
       [1033,   23],
       [ 896,   20],
       [ 849,   20],
       [ 458,   19],
       [  45,   17],
       [ 482,   17],
       [ 617,   17],
       [ 626,   17],
       [ 929,   17],
       [1233,   17],
       [  50,   16],
       [ 154,   16],
       [ 178,   16],
       [ 285,   16],
       [ 336,   16],
       [ 685,   16],
       [1411,   16],
       [1527,   16],
       [1548,   16],
       [ 433,   16],
       [ 958,   16],
       [ 967,   16],
       [ 204,   16],
       [ 999,   16],
       [  82,   15],
       [ 118,   15],
       [ 136,   15],
       [ 302,   15],
       [ 305,   15],
       [ 356,   15],
       [ 519,   15],
       [ 634,   15],
       [ 699,   15],
       [ 977,   15],
       [1006,   15],
       [1275,   15],
       [1282,   15],
       [1299,   15],
       [1385,   15],
       [1416,   15],
       [1420,   15],
       [1473,   15],
       [  69,

In [83]:
logmodel.fit(X_train[:,sorted_common_index_n_runs[:, 0]],Y_train)         
predictions = logmodel.predict(X_test[:,sorted_common_index_n_runs[:, 0]])
accuracy_score(Y_test,predictions)

0.836